In [ ]:
import pandas as pd
import numpy as np

# 1. Configurações Visuais
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

# 2. Carregamento dos Dados
caminho_arquivo = '../../../planilhas/limpo/modulo_3/limpar_e_padronizar_colunas/inscritos_limpo.csv'
try:
    df = pd.read_csv(caminho_arquivo)
except FileNotFoundError:
    df = pd.read_csv('inscritos_limpo.csv')

# 3. Filtros (Cenário Real)
df = df[(df['opcao_curso'] == 1) & (df['modalidade_fies'] == 'Modalidade I')]

# 4. Tratamento de Área
df['nome_cine_area_geral'] = df['nome_cine_area_geral'].fillna('Outras Áreas')

# 5. Faixas de Renda (Mantendo a estrutura de 6 faixas)
bins_renda = [0, 600, 1200, 1800, 2400, 3000, 10000]
labels_renda = [
    '1. Renda 0 - 600', 
    '2. Renda 601 - 1200', 
    '3. Renda 1201 - 1800', 
    '4. Renda 1801 - 2400', 
    '5. Renda 2401 - 3000', 
    '6. Renda > 3000'
]
df['Faixa Renda'] = pd.cut(df['renda_per_capita'], bins=bins_renda, labels=labels_renda)

# 6. DEFINIÇÃO DOS 6 NÍVEIS DE NOTA (Escala Expressiva de 150 pontos)
df['gap'] = df['media_enem'] - df['nota_corte_gp']

# Intervalos: 
# < -150 | -150 a -50 | -50 a 0 | 0 a 50 | 50 a 150 | > 150
bins_gap = [-1000, -150, -50, 0, 50, 150, 1000]

labels_gap = [
    '6. Nota Muito Inferior ao Corte (Gap < -150)',
    '5. Nota Inferior ao Corte (Gap -150 a -50)',
    '4. Nota Pouco Inferior ao Corte (Gap -50 a 0)',
    '3. Nota Pouco Superior ao Corte (Gap 0 a +50)',
    '2. Nota Superior ao Corte (Gap +50 a +150)',
    '1. Nota Muito Superior ao Corte (Gap > +150)'
]

df['Nivel_Nota'] = pd.cut(df['gap'], bins=bins_gap, labels=labels_gap)

# 7. Classificação de Status Formal
def classificar_status(status):
    if status == 'CONTRATADA':
        return 'Contratado'
    elif status in ['NÃO CONTRATADO', 'REJEITADA PELA CPSA', 'PARTICIPACAO CANCELADA PELO CANDIDATO', 'INSCRIÇÃO POSTERGADA']:
        return 'Pré-Selecionado (Não Contratou)'
    elif status == 'LISTA DE ESPERA':
        return 'Lista de Espera'
    else:
        return 'Outros'

df['Status_Resumo'] = df['situacao_fies'].apply(classificar_status)

# 8. Agrupamento Final (Área -> Renda -> Nível Nota Expressivo)
analise_6_niveis = df.groupby(
    ['nome_cine_area_geral', 'Faixa Renda', 'Nivel_Nota'], 
    observed=True
).agg(
    # Quantidade
    Total_Inscritos=('id_estudante', 'count'),
    
    # Destino dos Candidatos
    Qtd_Contratados=('Status_Resumo', lambda x: (x == 'Contratado').sum()),
    Qtd_Pre_Selecionados_Nao_Contrataram=('Status_Resumo', lambda x: (x == 'Pré-Selecionado (Não Contratou)').sum()),
    Qtd_Lista_Espera=('Status_Resumo', lambda x: (x == 'Lista de Espera').sum()),
    
    # Indicadores Financeiros
    Media_Perc_Financiamento=('percentual_financiamento', 'mean'),
    Media_Semestres=('qtde_semestre_financiado', 'mean')

).reset_index()

# 9. Limpeza (Remover linhas vazias)
analise_6_niveis = analise_6_niveis[analise_6_niveis['Total_Inscritos'] > 0]

# Renomear colunas para o relatório final
analise_6_niveis.columns = [
    'Área do Conhecimento', 
    'Faixa de Renda', 
    'Nível da Nota (Gap)', 
    'Total Candidatos', 
    'Qtd. Contratados', 
    'Qtd. Barrados Burocracia', 
    'Qtd. Lista Espera', 
    'Média % Financiamento', 
    'Média Semestres'
]

# Exibição
analise_6_niveis

<ipython-input-1-5c646669f587>:14: DtypeWarning: Columns (25,37,38,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo)


,Área do Conhecimento,Faixa de Renda,Nível da Nota (Gap),Total Candidatos,Qtd. Contratados,Qtd. Barrados Burocracia,Qtd. Lista Espera,Média % Financiamento,Média Semestres
0,"Agricultura, silvicultura, pesca e veterinária",1. Renda 0 - 600,6. Nota Muito Inferior ao Corte (Gap < -150),640,55,118,460,88.26,9.33
1,"Agricultura, silvicultura, pesca e veterinária",1. Renda 0 - 600,5. Nota Inferior ao Corte (Gap -150 a -50),4747,610,1223,2867,88.68,9.22
2,"Agricultura, silvicultura, pesca e veterinária",1. Renda 0 - 600,4. Nota Pouco Inferior ao Corte (Gap -50 a 0),3684,1161,1858,609,89.05,9.45
3,"Agricultura, silvicultura, pesca e veterinária",1. Renda 0 - 600,3. Nota Pouco Superior ao Corte (Gap 0 a +50),2252,1050,1108,52,89.29,9.59
4,"Agricultura, silvicultura, pesca e veterinária",1. Renda 0 - 600,2. Nota Superior ao Corte (Gap +50 a +150),1110,469,604,17,89.19,9.56
5,"Agricultura, silvicultura, pesca e veterinária",1. Renda 0 - 600,1. Nota Muito Superior ao Corte (Gap > +150),35,13,22,0,88.62,9.77
6,"Agricultura, silvicultura, pesca e veterinária",2. Renda 601 - 1200,6. Nota Muito Inferior ao Corte (Gap < -150),1288,72,280,926,79.02,8.96
7,"Agricultura, silvicultura, pesca e veterinária",2. Renda 601 - 1200,5. Nota Inferior ao Corte (Gap -150 a -50),8687,840,2586,5195,77.85,8.92
8,"Agricultura, silvicultura, pesca e veterinária",2. Renda 601 - 1200,4. Nota Pouco Inferior ao Corte (Gap -50 a 0),5829,1286,3440,1038,76.69,9.03
9,"Agricultura, silvicultura, pesca e veterinária",2. Renda 601 - 1200,3. Nota Pouco Superior ao Corte (Gap 0 a +50),3120,1005,1934,130,76.87,9.16
